# Daily workflow - troselab (DRAFT)

`TR TODO: Jupyter ipywidget GUI`

(check Daniel's ipysheet mouse entry GUI for inspiration)

1. Upload to analysis server
    1. automatic (rcopy with file consolidation)?
    2. WinSCP?

    
2. Session ingest - Have this script parse the user folder. 
    1. retrieve login name  **DONE**
    2. parse user storage folder for SessionIDs, ScanIDs **DONE**
    3. intersect with 'Session' tables **DONE**
    4. FIRST STEP: session ingestion of new data  **DONE**
        1. have ipywidget with list of data to be ingested with checkbox  **DONE**
        2. have dropdowns next to SessionsFetch that expose 'Project', 'Anatomical Location', 'Equipment' (NOTE: have me or admin accept new entries? Or user?) - have a user default **DONE**
            1. ***TODO*** include a mini2p settings shorthand in elements `imaging.ProcessingParamSet`.
        3. IMPORTANT: Have dropdown for `Same_Location_as_Session` - default: same session
            1. Include `Same_Location_as_Session` key in `Session` table (default same session)
            2. Update after Session ingest
        4. have central 'START' button **DONE**
        5. Combine session entry with automatic RSpace entry (**ongoing***)
            0. Include rspace in install requirements!
            1. Extract parent folder IDs under 'experiments' from RSpace (add RSpace API key and URL to djconfig )
            2. Intersect list with animal ID from session ingest
            3. Create new document with `YYYY-MM-DD_SessionID`
            4. Append content as verbose tables using Pandas df.to_html (`html_table = df.to_html()`)
            5. IMPORTANT: Push RSpace document ID and noteobook URL in Sessions database!
        5. OPTIONAL: Have a notebook section to enter and insert Session Notes
        6. OPTIONAL: Submit Sessions to pyrat. Not sure if necessary.
        7. TODO: Think if splitting projects off to different database suffixes (and differnt associated tables makes sense from here. Seems reasonable given the focus on the user root directory here.
        8. TODO: Implement auto animal pyrat ingest if needed. Crashes now.
        9. TODO: Make sure that multi-scan sessions are properly treated. Currently they are not detected as such by the GUI. Make sure ingestion behavior is appropriate.



3. Scan ingest / 1st stage s2p processing: 
    1. have another list with checkboxes (default checled) - now with all first stage scans of the newly ingested sessions. Have dropdown boxes with s2p parameteres. User default is selected. Followed by 'START'
    2. TODO: suppress s2p output, just have progress bars
    3. OPTIONAL: Have a notebook section to enter and insert Scan Notes
    4. OPTIONAL: Submit sessions, scans and Notes to RSPACE! Upload s2p average image, max image, Source image and Rastermap 
    

4. Scan Curation
    1. DIFFERENT NOTEBOOK (or different Notebook section)
    2. Make intersection of folder with database
        1. list _all_ scans
        2. have all non-curated red - the others green
        3. bhave non-processed scans grey
        4. have 'CURATE LOCALLY WITH s2P' buttons behind all scans. This button should spawn a local suite2p GUI which directly loads the respective suite2p stats.npy file from the server. Folder settings for server need to be set in datajoint.json file.

5. SERVER: Database backup and Data folder backup
    1. Have cronjob backup database to ana2 and isilon (midnight, incremental - only add, no delete. Increment ID to be reversible?)
    2. Have cronjob backup data folders to ana2 and isilon (midnight, incremental - only add, no delete)

## Login

Either log in via a local config file (see [01_pipeline](./01_pipeline.ipynb)), or enter login information manually. If you are don't have your login information, contact the administrator.


In [1]:
import os
# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='adamacs', ("Please move to the main directory")
from adamacs.pipeline import subject, session, equipment, surgery 
from adamacs.ingest import session as isess
import datajoint as dj
from rspace_client.eln import eln
import pandas as pd


[2023-03-17 12:05:29,011][INFO]: Connecting tobiasr@172.26.128.53:3306
[2023-03-17 12:05:29,068][INFO]: Connected tobiasr@172.26.128.53:3306


### RSpace connection

In [7]:
URL=dj.config['custom'].get('rspace_URL')
API_KEY=dj.config['custom'].get('rspace_API_key')
api = eln.ELNClient(URL, API_KEY)
api.get_status()

{'message': 'OK', 'rspaceVersion': '1.80.1'}

### write an example document

In [ ]:
new_doc = api.create_document(name=f"Analysis of dataset {animalID}", parent_folder_id=matching_folder_id)

In [ ]:
new_doc = api.create_document(name=f"Analysis of dataset {animalID}", parent_folder_id=matching_folder_id)
content = f"""
<p>Analysis of temperature dataset from our standard locations.
<p>No variation between locations:
Raw data: <fileId={animalID}>
<p>
Statistical summary: <fileId={animalID}>b
<p>
Location vs temperature: <fileId={animalID}>
"""
api.append_content(new_doc['id'], content)
documents['documents'][0]['parentFolderId']

## Activation
Next, import from `adamacs.pipeline` to activate the relevant schema.

In [2]:
from adamacs.utility import *
from adamacs.nbgui import *
from adamacs.pipeline import subject, session, surgery, scan, equipment

Assign easy names for relevant tables

In [3]:
sub, lab, protocol, line, mutation, user, project, subject_genotype, subject_death = (
    subject.Subject(), subject.Lab(), subject.Protocol(), subject.Line(), 
    subject.Mutation(), subject.User(), subject.Project(), subject.SubjectGenotype(), 
    subject.SubjectDeath()
    )

## 1. Data upload
(not implemented currently)

## 2. Session ingest

### Define ingest GUI functions

In [4]:
# Tobias Rose 2023: Routine ingest helpers

import ipywidgets as widgets
from natsort import natsorted, ns
import re
from tqdm import tqdm

def select_sessions(IngestedSessionDirA, AvailableSessionDirB, Project, Recording_Location, Equipment, s2pparm, SessionNotes):
    
    # Personal default values
    user_defaults = get_user_defaults(AvailableSessionDirB)
       
    # Define the widgets
    session_dropdowns = []
    session_checkboxes = []
    for i, session_list in enumerate(AvailableSessionDirB):
        # Dropdowns for Project, Recording Location, Equipment

        current_session = get_session_key_from_dir([session_list])

        # check if directory session is already ingested - if yes: populate GUI with table values. If no: use user defaults
        query = session.Session() & f'session_id = "{current_session[0]}"'

        count = len(query.fetch('session_id'))
        print(current_session)
        
        if count > 0:
            # print("HELLO")
            # get the project associated with a session
            query = session.ProjectSession() & f'session_id = "{current_session[0]}"'
            project_dropdown_value = query.fetch("project")
            # get the location associated with a session
            query = session.Session() * scan.ScanLocation() & f'session_id = "{current_session[0]}"'
            location_dropdown_value = query.fetch("anatomical_location")
            # get the equipment associated with a session
            query = session.Session() * scan.Scan() & f'session_id = "{current_session[0]}"'
            equipment_dropdown_value = query.fetch("scanner")
            # get the note associated with a session
            query = session.SessionNote() & f'session_id = "{current_session[0]}"'
            session_note_textbox_value = query.fetch("session_note")
            if len(session_note_textbox_value) == 0:
                session_note_textbox_value = ["none"]    
        else:
            project_dropdown_value = Project[user_defaults[i][0]]
            location_dropdown_value = [Recording_Location[user_defaults[i][1]]]
            equipment_dropdown_value = [Equipment[user_defaults[i][2]]]
            session_note_textbox_value = ["none"]
        
        project_dropdown = widgets.Dropdown(options=Project, value=project_dropdown_value, description="Project:")
        location_dropdown = widgets.Dropdown(options=Recording_Location, value=location_dropdown_value[0], description="Location:")
        equipment_dropdown = widgets.Dropdown(options=Equipment, value=equipment_dropdown_value[0], description="Setup:")
        s2pparms_dropdown = widgets.Dropdown(options=s2pparm, value=s2pparm[user_defaults[i][3]], description="s2p parm:")
        session_note_textbox = widgets.Text(value=session_note_textbox_value[0], description='Session comment:')

        session_dropdowns.append((project_dropdown, location_dropdown, equipment_dropdown, s2pparms_dropdown, session_note_textbox))

        # Checkbox for Process?
        session_checkbox = widgets.Checkbox(description='run?', layout=widgets.Layout(width='auto'))
        session_checkboxes.append(session_checkbox)

    # Display the widgets
    output = widgets.Output()

    with output:
        # Display the Sessions labels and associated dropdowns and checkboxes
        hbox_list = []
        for i, session_list in enumerate(AvailableSessionDirB): #unique_directory_strings(SessionDirA, SessionDirB)
            # Create an HBox to hold the label and associated dropdowns and checkbox
            hbox = widgets.HBox()
            hbox.children = [
                widgets.Label(value=session_list + ':', layout=widgets.Layout(width='1800px')), 
                session_dropdowns[i][0],
                session_dropdowns[i][1],
                session_dropdowns[i][2],
                session_dropdowns[i][3],
                session_dropdowns[i][4],
                session_checkboxes[i]
            ]
            if session_list in unique_directory_strings(IngestedSessionDirA, AvailableSessionDirB):
                hbox.children[0].value = '*' + session_list + ':'
                hbox.children[6].value = True
            else:
                hbox.children[6].value = False
                hbox.children[1].disabled = True
                hbox.children[2].disabled = True
                hbox.children[3].disabled = True
                hbox.children[4].disabled = True
                hbox.children[5].disabled = True
            hbox_list.append(hbox)

        vbox = widgets.VBox(hbox_list, layout=widgets.Layout(flex='0 0 auto', overflow_y='scroll'))
        # Display the commit button
        commit_button = widgets.Button(description='Commit', layout=widgets.Layout(width='auto'))
        display(vbox, commit_button)

        # Define the callback function for the commit button
        def commit_button_clicked(b):
            selected_sessions = [AvailableSessionDirB[i] for i in range(len(AvailableSessionDirB)) if session_checkboxes[i].value]
            selected_projects = [session_dropdowns[i][0].value for i in range(len(AvailableSessionDirB)) if session_checkboxes[i].value]
            selected_locations = [session_dropdowns[i][1].value for i in range(len(AvailableSessionDirB)) if session_checkboxes[i].value]
            selected_equipment = [session_dropdowns[i][2].value for i in range(len(AvailableSessionDirB)) if session_checkboxes[i].value]
            selected_s2pparms = [session_dropdowns[i][3].value for i in range(len(AvailableSessionDirB)) if session_checkboxes[i].value]
            entered_session_note = [session_dropdowns[i][4].value for i in range(len(AvailableSessionDirB)) if session_checkboxes[i].value]

            selected_scans = get_scan_key_from_dir(selected_sessions)
            selected_sessions = get_session_key_from_dir(selected_sessions)

            output.clear_output()
            with output:
                # print('Selected Sessions:', selected_sessions)
                # print('Selected Scans:', selected_scans)
                # print('Selected Projects:', selected_projects)
                # print('Selected Recording Locations:', selected_locations)
                # print('Selected Equipment:', selected_equipment)
                # print('Selected S2PParms:', selected_s2pparms)

                # Ingest selected sessions here
                populate_settings = {'display_progress': False}
                for i, sessi in enumerate(tqdm(selected_sessions, desc='Current Session: {}')):
                    # ingest sessions
                    isess.ingest_session_scan(sessi, verbose=False, project_key=selected_projects[i], equipment_key=selected_equipment[i], location_key=selected_locations[i], software_key='ScanImage')
                    # update / insert session info based on user choice from above
                    try:
                        session.SessionNote.insert1({'session_id': sessi, 'session_note': entered_session_note[i]})
                    except:
                        session.SessionNote.delete_quick({'session_id': sessi})
                        session.SessionNote.insert1({'session_id': sessi, 'session_note': entered_session_note[i]})
                    
                    # update scaninfo based on user choice from above
                    # get the scans associated with a session
                    query = session.Session() * scan.Scan() & f'session_id = "{sessi}"'
                    scans_to_process = query.fetch("scan_id")
                    
                    for j, scansi in enumerate(scans_to_process):
                        try:
                            scan.Scan.update1({'session_id': sessi, 'scan_id': scansi, 'scan_notes': entered_session_note[i]})
                        except:
                            scan.Scan.insert1({'session_id': sessi, 'scan_id': scansi, 'scan_notes': entered_session_note[i]})
                        try:
                            scan.ScanLocation.update1({'session_id': sessi, 'scan_id': scansi, 'anatomical_location': selected_locations[i]})
                        except: 
                            scan.ScanLocation.insert1({'session_id': sessi, 'scan_id': scansi, 'anatomical_location': selected_locations[i]})
                        
                        # print('Ingested and processed Scan:', scansi)
                print('Ingested Sessions:', selected_sessions)

                # TODO: Queue Processing here


                ## Make RSpace entries here
                query = session.Session() & f'session_id = "{sessi}"'
                animalID = query.fetch("scan_id")
                date = query.fetch("scan_id")
                make_rspace_session_document(animalID, sessi, date) # TODOO
                
                
                return selected_sessions

        # Attach the callback function to the commit button
        commit_button.on_click(commit_button_clicked)

    # Display the output
    display(output)
    
def make_rspace_session_document(animalID, sessionID, date):
    
    # find 'Experiments' folder and get ID
    folders = api.list_folder_tree()
    experiments_ids = []
    for record in folders['records']:
        if record['name'] == 'Experiments':
            experiments_ids.append(record['id'])
    
    # find folders under 'Experiments' folder
    names_and_ids = []
    subfolders = api.list_folder_tree(experiments_ids[0])
    names_and_ids.append([(record['name'], record['id']) for record in subfolders['records']])

    # find matches with animal IDs
    matching_folder_id = [x[1] for sublist in names_and_ids for x in sublist if animalID in x[0]][0]

    # create RSpace document
    new_doc = api.create_document(name=f"{date}_{sessionID}", parent_folder_id=matching_folder_id)
    
    fetchtable = session.Session() * session.SessionDirectory * session.SessionUser * session.ProjectSession * session.SessionNote 
    df = fetchtable.fetch(format='frame')
    html_table = df.to_html()
    content = html_table
    api.append_content(new_doc['id'], content)
        



def get_date_key_from_dir(directory):
    return directory.split("_")[-1]

def get_session_dir_key_from_dir(directory):
    return [path.split('/')[-1] for path in directory]
     
def get_scan_dir_key_from_dir(directory):
    return [path.split('/')[-1] for path in directory]

def get_session_key_from_dir(string):
    result = [re.search(r'sess\S+', item).group(0) for item in string]
    return result

def get_user_initials_from_dir(string):
    result = [name[:2] for name in string]
    return result

def get_scan_key_from_dir(string):
    result = [re.search(r'scan\S+_', item).group(0)[:-1] for item in string]
    return result

def unique_directory_strings(dirs1, dirs2):
    set1 = set(dirs1)
    set2 = set(dirs2)
    common_dirs = list(set1.intersection(set2))
    unique_dirs = list(set(set1.union(set2)) - set(common_dirs))
    return unique_dirs

def get_user_defaults(directory):
    RN = [1, 1, 2, 0]
    JJ = [2, 6, 2, 0]
    TR = [0, 6, 3, 0]
    DB = [4, 0, 1, 2]
    NK = [5, 0, 2, 2]
    
    user_initials = get_user_initials_from_dir(directory)

    user_arrays = {}
    for initial in user_initials:
        values = locals()[initial]
        user_arrays[initial] = values

    new_array = [user_arrays[initial] for initial in user_initials]
    return new_array

In [ ]:
session.Session().delete()

In [41]:
query = session.Session() * subject.User() & f'session_id = "sess9FGTSPEH"'
animalID = query.fetch("subject")[0]
date = query.fetch("session_datetime")[0].strftime("%Y-%m-%d")
userID = query.fetch("initials")[0]
sessionID = query.fetch("session_id")[0]
# make_rspace_session_document(animalID, "sess9FGTSPEH", date) # TODOO

In [56]:
session.Session() * scan.ScanPath() * session.SessionUser() * session.ProjectSession() * session.SessionNote() * scan.Scan() 

session_id,scan_id,user_id,path,project,subject PyRat import uses this for earmark value,session_datetime,session_note,scanner,acq_software,scan_notes free-notes
sess9FG1TAXY,scan9FG1TAXY,1,/datajoint-data/data/tobiasr/TR_WEZ-8701_2023-01-13_scan9FG1TAXY_sess9FG1TAXY,ATn,WEZ-8701,2023-01-13 00:00:00,Test,mini2p_01,ScanImage,Test
sess9FG1TAXY,scan9FG1THFD,1,/datajoint-data/data/tobiasr/TR_WEZ-8701_2023-01-13_scan9FG1THFD_sess9FG1TAXY,ATn,WEZ-8701,2023-01-13 00:00:00,Test,mini2p_01,ScanImage,Test


In [58]:
scan.subject()

TypeError: 'module' object is not callable

In [96]:
# def make_rspace_session_document(animalID, sessionID, date):

# find 'Experiments' folder and get ID
folders = api.list_folder_tree()
experiments_ids = []
for record in folders['records']:
    if record['name'] == 'Experiments':
        experiments_ids.append(record['id'])

# find folders under 'Experiments' folder
names_and_ids = []
subfolders = api.list_folder_tree(experiments_ids[0])
names_and_ids.append([(record['name'], record['id']) for record in subfolders['records']])

# find matches with animal IDs
match_found = []
match_found.append(any(animalID in item[0] for sublist in names_and_ids for item in sublist))

# if found, find folder id matching that animal
if any(match_found) == True:
    save_folder_id = [x[1] for sublist in names_and_ids for x in sublist if animalID in x[0]][0]
else:
    # if not generate the directory
    new_folder = api.create_folder(name=f"{userID}_{animalID}", parent_folder_id=experiments_ids[0], notebook=True)
    save_folder_id = new_folder['id']

# create RSpace document
# find matches with document IDs
# find all documents in the animal folder
all_animal_documents = api.list_folder_tree(save_folder_id)

match_found = []
match_found.append(any(sessionID in item[0] for sublist in all_animal_documents for item in sublist))

if any(match_found) == True:
    # find document id
    doc_names_and_ids = []
    doc_names_and_ids.append(record['id'] for record in all_animal_documents['records'])
    new_doc = {"id": doc_names_and_ids[0][0]}
else:
    new_doc = api.create_document(name=f"{date}_{sessionID}", parent_folder_id=save_folder_id)

fetchtable = session.Session() * scan.ScanPath() * session.SessionUser() * session.ProjectSession() * session.SessionNote() * scan.Scan() 
df = fetchtable.fetch(format='frame')
html_table = df.to_html()
content = html_table
api.append_content(new_doc['id'], content)

{'id': 104955,
 'globalId': 'SD104955',
 'name': '2023-03-01_sess9FGTSPEH',
 'created': '2023-03-17T13:00:05.411Z',
 'lastModified': '2023-03-17T13:00:06.383Z',
 'parentFolderId': 104919,
 'signed': False,
 'tags': None,
 'form': {'id': 5,
  'globalId': 'FM5',
  'stableId': '1596118135085null',
  'version': 0,
  'name': 'Basic Document',
  'tags': None,
  'formState': 'PUBLISHED',
  'accessControl': {'ownerPermissionType': 'WRITE',
   'groupPermissionType': 'NONE',
   'worldPermissionType': 'READ'},
  'iconId': -1,
  '_links': [{'link': 'https://rspace.rhrz.uni-bonn.de/api/v1/forms/5',
    'rel': 'self'}]},
 'owner': {'id': 491534,
  'username': 'trose',
  'email': 'trose@uni-bonn.de',
  'firstName': 'Tobias',
  'lastName': 'Rose',
  'homeFolderId': 23780,
  'workbenchId': None,
  '_links': []},
 'fields': [{'id': 2425795,
   'globalId': 'FD2425795',
   'name': 'Data',
   'type': 'text',
   'content': '<table border="1" class="dataframe"> \n <thead> \n  <tr style="text-align: right;"> 

In [88]:
doc_names_and_ids = []
doc_names_and_ids.append([record['id'] for record in all_animal_documents['records']])
new_doc = {"id": doc_names_and_ids[0][0]}

In [89]:
api.list_folder_tree(save_folder_id)

{'totalHits': 3,
 'pageNumber': 0,
 '_links': [{'link': 'https://rspace.rhrz.uni-bonn.de/api/v1/folders/tree?pageNumber=0',
   'rel': 'self'}],
 'parentId': 67345,
 'records': [{'id': 104940,
   'globalId': 'SD104940',
   'name': '2023-03-01_sess9FGTSPEH',
   'created': '2023-03-17T12:52:39.143Z',
   'lastModified': '2023-03-17T12:52:40.175Z',
   'parentFolderId': 104919,
   'type': 'DOCUMENT',
   '_links': [{'link': 'https://rspace.rhrz.uni-bonn.de/api/v1/documents/104940',
     'rel': 'self'}],
   'owner': {'id': 491534,
    'username': 'trose',
    'email': 'trose@uni-bonn.de',
    'firstName': 'Tobias',
    'lastName': 'Rose',
    'homeFolderId': 23780,
    'workbenchId': None,
    '_links': []}},
  {'id': 104938,
   'globalId': 'SD104938',
   'name': 'Untitled strip',
   'created': '2023-03-17T12:51:47.951Z',
   'lastModified': '2023-03-17T12:52:00.820Z',
   'parentFolderId': 104919,
   'type': 'DOCUMENT',
   '_links': [{'link': 'https://rspace.rhrz.uni-bonn.de/api/v1/documents/10

In [9]:
# find 'Experiments' folder and get ID
folders = api.list_folder_tree()
experiments_ids = []
for record in folders['records']:
    if record['name'] == 'Experiments':
        experiments_ids.append(record['id'])

names_and_ids = []
subfolders = api.list_folder_tree(experiments_ids[0])
names_and_ids.append([(record['name'], record['id']) for record in subfolders['records']])


# find matches with animal IDs
animalID = 'ROS-6666'

partial_matches = [x for sublist in names_and_ids for x in sublist if animalID in x[0]]
partial_matches_unique = set(partial_matches)

matching_folder_id = [x[1] for sublist in names_and_ids for x in sublist if animalID in x[0]][0]

print(partial_matches)
print(partial_matches_unique)
print(matching_folder_id)

partial_matches_dict = {'animalID': animalID, 'ID': [x[1] for x in partial_matches_unique][0]}


print(partial_matches_dict)



[('TR_ROS-6666', 67346)]
{('TR_ROS-6666', 67346)}
67346
{'animalID': 'ROS-6666', 'ID': 67346}


In [ ]:
matching_folmatching_folder_id = [x[1] for sublist in names_and_ids[0] for x in sublist if animalID in x[0]][0]

In [ ]:
session_note_textbox_value = ["none"]
session_note_textbox_value[0]

In [ ]:
current_session = ["sess9FGTSPEH"]
query = session.SessionNote() & f'session_id = "{current_session[0]}"'
session_note_textbox_value = query.fetch("session_note")
session_note_textbox_value

#### Populate ingest GUI variables from DJ tables and manual arrays

In [ ]:
# DJ tables
Project = project.fetch('project')
Equipment = equipment.Equipment().fetch('scanner')
Recording_Location = surgery.AnatomicalLocation().fetch('anatomical_location')

session_dirs_ingested = session.SessionDirectory.fetch('session_dir')
scan_dirs_ingested = scan.ScanPath.fetch('path')
SessionsDirArrayingested = get_session_dir_key_from_dir(session_dirs_ingested)
ScanDirArrayingested = get_scan_dir_key_from_dir(scan_dirs_ingested)

SessionNotes = session.SessionNote.fetch('session_note')

# Manual arrays
s2pparm = ['mini2p_a', 'mini2p_b', 'bench2p_a']



In [ ]:
Recording_Location = surgery.AnatomicalLocation().fetch('anatomical_location')
Recording_Location


#### Parse user data root dir and sort by date-string in directory name

In [ ]:
dataroot = dj.config['custom']['exp_root_data_dir'][0]
dirs_root = [d for d in os.listdir(dataroot) if os.path.isdir(os.path.join(dataroot, d)) and 'sess' in d]

sorted_dirs_root = natsorted(dirs_root, key=get_date_key_from_dir, reverse = True)

In [ ]:
animalID = query.fetch("scan_id")

### Generate Ingest GUI

In [ ]:
# print("\033[1m" + 'Select:                                           Project        Location  Setup        s2p parm' + "\033[0m" )
print("\033[1m" + 'INGEST GUI' + "\033[0m" )
selected_data = select_sessions(SessionsDirArrayingested, sorted_dirs_root, Project, Recording_Location, Equipment, s2pparm, SessionNotes)

In [ ]:
get_scan_key_from_dir(['JJ_ROS-1438_2022-11-22_16_09scan9FF6TT76_sess9FF6TT7'])

In [ ]:
session.Session().delete(prompt = None)

In [ ]:
session.Session()

selected_data

In [ ]:
sessi = "sess9FG1TAXY"
query = session.Session() * scan.Scan() & f'session_id = "{sessi}"'
scans_to_process = query.fetch("scan_id")
for j, scansi in enumerate(scans_to_process):
    print(scansi)

In [ ]:
scans_to_process

In [ ]:
subject.Subject.delete()

### 2.1 Session notes ingest

In [ ]:
current_session = "sess9FGTSPEH"
query = session.Session() * scan.Scan() & f'session_id = "{current_session}"'
equipment_dropdown_value = query.fetch("scanner")
equipment_dropdown_value

## 3. Scan ingest / s2p processing 1st pass

In [ ]:
query = session.Session() * scan.ScanLocation() & f'session_id = "{current_session}"'

In [ ]:
query